In [1]:
import pickle
import numpy as np
from keras.models import Model
from keras.layers import Input, Lambda, Reshape, LSTM, Dense
from keras.optimizers import Adam
from keras.utils import to_categorical
from keras import backend as K

Using TensorFlow backend.


In [9]:
tokenized_text, char_to_idx, idx_to_char = pickle.load(open('../data/processed-alice.pickle', 'rb'))
X, Y = pickle.load(open('../data/alice-x-y-data.pickle', 'rb'))

Y = np.reshape(Y, (X.shape[1], X.shape[0], X.shape[2]))

# Global Variables
m, Tx, n_values = X.shape
n_a = 64 # for LSTM with 64-dimensional hidden states

# Train the Model
First we train the model to get the proper weights.
Remember, we must declare the layers in our model as global variables so they
can be reused in the Text Generation step

In [10]:
# Reused Layer for Text Generation: Reshape, LSTM, Dense
reshapor = Reshape((1, n_values)) # makes input 3-dimensional
LSTM_cell = LSTM(n_a, return_state=True)
densor = Dense(n_values, activation='softmax')

In [11]:
def learn_to_talk(Tx, n_a, n_values):
    X = Input(shape=(Tx, n_values))
    
    # Define initial hidden state for LSTM
    a0 = Input(shape=(n_a,), name='a0')
    c0 = Input(shape=(n_a,), name='c0')
    a = a0
    c = c0
    
    outputs = []
    for t in range(Tx):
        # Get t'th vector
        x = Lambda(lambda x: x[:,t,:])(X)
        # Reshape x to be (1,Tx)
        x = reshapor(x)
        # Perform 1 step of LSTM
        a, _, c = LSTM_cell(x)
        # apply densor to hidden state output of LSTM_cell
        out = densor(a)
        outputs.append(out)
    return Model([X, a0, c0], outputs)

In [12]:
model = learn_to_talk(Tx, n_a, n_values)

In [13]:
opt = Adam(lr=0.001, beta_1=0.9, beta_2=0.999, decay=0.0005)
model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])

In [14]:
# a0 = np.zeros((m, n_a))
# c0 = np.zeros((m, n_a))

a0 = np.random.rand(m, n_a)
c0 = np.random.rand(m, n_a)

In [15]:
model.fit([X, a0, c0], list(Y), epochs=30)

Epoch 1/30
1031/1031 [==============================] - 197s 191ms/step - loss: 527.8116 - dense_2_loss: 3.7721 - dense_2_acc: 0.1329 - dense_2_acc_1: 0.1300 - dense_2_acc_2: 0.1339 - dense_2_acc_3: 0.1164 - dense_2_acc_4: 0.1377 - dense_2_acc_5: 0.1280 - dense_2_acc_6: 0.1377 - dense_2_acc_7: 0.1426 - dense_2_acc_8: 0.1620 - dense_2_acc_9: 0.1242 - dense_2_acc_10: 0.1339 - dense_2_acc_11: 0.1785 - dense_2_acc_12: 0.1523 - dense_2_acc_13: 0.1435 - dense_2_acc_14: 0.1397 - dense_2_acc_15: 0.1620 - dense_2_acc_16: 0.1280 - dense_2_acc_17: 0.1445 - dense_2_acc_18: 0.1261 - dense_2_acc_19: 0.1435 - dense_2_acc_20: 0.1300 - dense_2_acc_21: 0.1426 - dense_2_acc_22: 0.1397 - dense_2_acc_23: 0.1290 - dense_2_acc_24: 0.1358 - dense_2_acc_25: 0.1290 - dense_2_acc_26: 0.1329 - dense_2_acc_27: 0.1726 - dense_2_acc_28: 0.2890 - dense_2_acc_29: 0.1416 - dense_2_acc_30: 0.1251 - dense_2_acc_31: 0.1358 - dense_2_acc_32: 0.1465 - dense_2_acc_33: 0.1309 - dense_2_acc_34: 0.1416 - dense_2_acc_35: 0.1309 

  64/1031 [>.............................] - ETA: 3s - loss: 493.6082 - dense_2_loss: 3.5543 - dense_2_acc: 0.1562 - dense_2_acc_1: 0.1719 - dense_2_acc_2: 0.1250 - dense_2_acc_3: 0.1562 - dense_2_acc_4: 0.2188 - dense_2_acc_5: 0.2656 - dense_2_acc_6: 0.2188 - dense_2_acc_7: 0.2344 - dense_2_acc_8: 0.2812 - dense_2_acc_9: 0.1875 - dense_2_acc_10: 0.2812 - dense_2_acc_11: 0.2969 - dense_2_acc_12: 0.2500 - dense_2_acc_13: 0.2031 - dense_2_acc_14: 0.2500 - dense_2_acc_15: 0.2344 - dense_2_acc_16: 0.2188 - dense_2_acc_17: 0.1875 - dense_2_acc_18: 0.1719 - dense_2_acc_19: 0.2188 - dense_2_acc_20: 0.2188 - dense_2_acc_21: 0.1719 - dense_2_acc_22: 0.1875 - dense_2_acc_23: 0.1562 - dense_2_acc_24: 0.2344 - dense_2_acc_25: 0.1719 - dense_2_acc_26: 0.1562 - dense_2_acc_27: 0.1250 - dense_2_acc_28: 0.3906 - dense_2_acc_29: 0.2031 - dense_2_acc_30: 0.2031 - dense_2_acc_31: 0.2031 - dense_2_acc_32: 0.1875 - dense_2_acc_33: 0.1562 - dense_2_acc_34: 0.2812 - dense_2_acc_35: 0.2812 - dense_2_acc_36: 0

1031/1031 [==============================] - 4s 4ms/step - loss: 477.4095 - dense_2_loss: 3.4241 - dense_2_acc: 0.1920 - dense_2_acc_1: 0.1940 - dense_2_acc_2: 0.1920 - dense_2_acc_3: 0.1843 - dense_2_acc_4: 0.1988 - dense_2_acc_5: 0.2017 - dense_2_acc_6: 0.1920 - dense_2_acc_7: 0.1901 - dense_2_acc_8: 0.2502 - dense_2_acc_9: 0.1882 - dense_2_acc_10: 0.1969 - dense_2_acc_11: 0.2561 - dense_2_acc_12: 0.2241 - dense_2_acc_13: 0.1950 - dense_2_acc_14: 0.1950 - dense_2_acc_15: 0.2328 - dense_2_acc_16: 0.1959 - dense_2_acc_17: 0.2008 - dense_2_acc_18: 0.1872 - dense_2_acc_19: 0.1901 - dense_2_acc_20: 0.1872 - dense_2_acc_21: 0.1969 - dense_2_acc_22: 0.1988 - dense_2_acc_23: 0.1775 - dense_2_acc_24: 0.1930 - dense_2_acc_25: 0.1911 - dense_2_acc_26: 0.1920 - dense_2_acc_27: 0.2541 - dense_2_acc_28: 0.4316 - dense_2_acc_29: 0.1901 - dense_2_acc_30: 0.1882 - dense_2_acc_31: 0.1930 - dense_2_acc_32: 0.1959 - dense_2_acc_33: 0.1950 - dense_2_acc_34: 0.1940 - dense_2_acc_35: 0.1920 - dense_2_acc_3

  64/1031 [>.............................] - ETA: 3s - loss: 432.8814 - dense_2_loss: 3.1613 - dense_2_acc: 0.2188 - dense_2_acc_1: 0.2812 - dense_2_acc_2: 0.2031 - dense_2_acc_3: 0.2344 - dense_2_acc_4: 0.2656 - dense_2_acc_5: 0.1875 - dense_2_acc_6: 0.1562 - dense_2_acc_7: 0.1406 - dense_2_acc_8: 0.3750 - dense_2_acc_9: 0.2344 - dense_2_acc_10: 0.2031 - dense_2_acc_11: 0.2812 - dense_2_acc_12: 0.2500 - dense_2_acc_13: 0.1562 - dense_2_acc_14: 0.1406 - dense_2_acc_15: 0.2344 - dense_2_acc_16: 0.1562 - dense_2_acc_17: 0.2188 - dense_2_acc_18: 0.0625 - dense_2_acc_19: 0.2031 - dense_2_acc_20: 0.2812 - dense_2_acc_21: 0.2344 - dense_2_acc_22: 0.1875 - dense_2_acc_23: 0.1719 - dense_2_acc_24: 0.1562 - dense_2_acc_25: 0.1562 - dense_2_acc_26: 0.1875 - dense_2_acc_27: 0.2500 - dense_2_acc_28: 0.5000 - dense_2_acc_29: 0.2188 - dense_2_acc_30: 0.1562 - dense_2_acc_31: 0.1250 - dense_2_acc_32: 0.1719 - dense_2_acc_33: 0.2344 - dense_2_acc_34: 0.3438 - dense_2_acc_35: 0.2344 - dense_2_acc_36: 0

1031/1031 [==============================] - 4s 4ms/step - loss: 428.0405 - dense_2_loss: 3.1020 - dense_2_acc: 0.1920 - dense_2_acc_1: 0.1940 - dense_2_acc_2: 0.1920 - dense_2_acc_3: 0.1843 - dense_2_acc_4: 0.1988 - dense_2_acc_5: 0.2017 - dense_2_acc_6: 0.1920 - dense_2_acc_7: 0.1901 - dense_2_acc_8: 0.2502 - dense_2_acc_9: 0.1882 - dense_2_acc_10: 0.1969 - dense_2_acc_11: 0.2561 - dense_2_acc_12: 0.2241 - dense_2_acc_13: 0.1950 - dense_2_acc_14: 0.1950 - dense_2_acc_15: 0.2328 - dense_2_acc_16: 0.1959 - dense_2_acc_17: 0.2008 - dense_2_acc_18: 0.1872 - dense_2_acc_19: 0.1901 - dense_2_acc_20: 0.1872 - dense_2_acc_21: 0.1969 - dense_2_acc_22: 0.1988 - dense_2_acc_23: 0.1775 - dense_2_acc_24: 0.1930 - dense_2_acc_25: 0.1911 - dense_2_acc_26: 0.1920 - dense_2_acc_27: 0.2541 - dense_2_acc_28: 0.4316 - dense_2_acc_29: 0.1901 - dense_2_acc_30: 0.1882 - dense_2_acc_31: 0.1930 - dense_2_acc_32: 0.1959 - dense_2_acc_33: 0.1950 - dense_2_acc_34: 0.1940 - dense_2_acc_35: 0.1920 - dense_2_acc_3

...

1031/1031 [==============================] - 4s 4ms/step - loss: 416.0388 - dense_2_loss: 3.0547 - dense_2_acc: 0.1920 - dense_2_acc_1: 0.1940 - dense_2_acc_2: 0.1920 - dense_2_acc_3: 0.1843 - dense_2_acc_4: 0.1988 - dense_2_acc_5: 0.2017 - dense_2_acc_6: 0.1920 - dense_2_acc_7: 0.1901 - dense_2_acc_8: 0.2502 - dense_2_acc_9: 0.1882 - dense_2_acc_10: 0.1969 - dense_2_acc_11: 0.2561 - dense_2_acc_12: 0.2241 - dense_2_acc_13: 0.1950 - dense_2_acc_14: 0.1950 - dense_2_acc_15: 0.2328 - dense_2_acc_16: 0.1959 - dense_2_acc_17: 0.2008 - dense_2_acc_18: 0.1872 - dense_2_acc_19: 0.1901 - dense_2_acc_20: 0.1872 - dense_2_acc_21: 0.1969 - dense_2_acc_22: 0.1988 - dense_2_acc_23: 0.1775 - dense_2_acc_24: 0.1930 - dense_2_acc_25: 0.1911 - dense_2_acc_26: 0.1920 - dense_2_acc_27: 0.2541 - dense_2_acc_28: 0.4316 - dense_2_acc_29: 0.1901 - dense_2_acc_30: 0.1882 - dense_2_acc_31: 0.1930 - dense_2_acc_32: 0.1959 - dense_2_acc_33: 0.1950 - dense_2_acc_34: 0.1940 - dense_2_acc_35: 0.1920 - dense_2_acc_3

  64/1031 [>.............................] - ETA: 3s - loss: 413.8618 - dense_2_loss: 3.1509 - dense_2_acc: 0.3125 - dense_2_acc_1: 0.1562 - dense_2_acc_2: 0.1875 - dense_2_acc_3: 0.1562 - dense_2_acc_4: 0.2500 - dense_2_acc_5: 0.2031 - dense_2_acc_6: 0.1406 - dense_2_acc_7: 0.2188 - dense_2_acc_8: 0.2969 - dense_2_acc_9: 0.1094 - dense_2_acc_10: 0.1406 - dense_2_acc_11: 0.2188 - dense_2_acc_12: 0.3125 - dense_2_acc_13: 0.2188 - dense_2_acc_14: 0.1719 - dense_2_acc_15: 0.2188 - dense_2_acc_16: 0.1406 - dense_2_acc_17: 0.2500 - dense_2_acc_18: 0.2031 - dense_2_acc_19: 0.1406 - dense_2_acc_20: 0.1719 - dense_2_acc_21: 0.1875 - dense_2_acc_22: 0.1562 - dense_2_acc_23: 0.1250 - dense_2_acc_24: 0.2188 - dense_2_acc_25: 0.1875 - dense_2_acc_26: 0.2500 - dense_2_acc_27: 0.3594 - dense_2_acc_28: 0.4219 - dense_2_acc_29: 0.2031 - dense_2_acc_30: 0.2344 - dense_2_acc_31: 0.2500 - dense_2_acc_32: 0.1875 - dense_2_acc_33: 0.2188 - dense_2_acc_34: 0.2344 - dense_2_acc_35: 0.2500 - dense_2_acc_36: 0

1031/1031 [==============================] - 4s 4ms/step - loss: 416.0039 - dense_2_loss: 3.0546 - dense_2_acc: 0.1920 - dense_2_acc_1: 0.1940 - dense_2_acc_2: 0.1920 - dense_2_acc_3: 0.1843 - dense_2_acc_4: 0.1988 - dense_2_acc_5: 0.2017 - dense_2_acc_6: 0.1920 - dense_2_acc_7: 0.1901 - dense_2_acc_8: 0.2502 - dense_2_acc_9: 0.1882 - dense_2_acc_10: 0.1969 - dense_2_acc_11: 0.2561 - dense_2_acc_12: 0.2241 - dense_2_acc_13: 0.1950 - dense_2_acc_14: 0.1950 - dense_2_acc_15: 0.2328 - dense_2_acc_16: 0.1959 - dense_2_acc_17: 0.2008 - dense_2_acc_18: 0.1872 - dense_2_acc_19: 0.1901 - dense_2_acc_20: 0.1872 - dense_2_acc_21: 0.1969 - dense_2_acc_22: 0.1988 - dense_2_acc_23: 0.1775 - dense_2_acc_24: 0.1930 - dense_2_acc_25: 0.1911 - dense_2_acc_26: 0.1920 - dense_2_acc_27: 0.2541 - dense_2_acc_28: 0.4316 - dense_2_acc_29: 0.1901 - dense_2_acc_30: 0.1882 - dense_2_acc_31: 0.1930 - dense_2_acc_32: 0.1959 - dense_2_acc_33: 0.1950 - dense_2_acc_34: 0.1940 - dense_2_acc_35: 0.1920 - dense_2_acc_3

  64/1031 [>.............................] - ETA: 3s - loss: 414.1338 - dense_2_loss: 3.2702 - dense_2_acc: 0.1875 - dense_2_acc_1: 0.1719 - dense_2_acc_2: 0.1562 - dense_2_acc_3: 0.1562 - dense_2_acc_4: 0.2188 - dense_2_acc_5: 0.1875 - dense_2_acc_6: 0.2656 - dense_2_acc_7: 0.1875 - dense_2_acc_8: 0.3438 - dense_2_acc_9: 0.2188 - dense_2_acc_10: 0.2812 - dense_2_acc_11: 0.2656 - dense_2_acc_12: 0.2344 - dense_2_acc_13: 0.1562 - dense_2_acc_14: 0.1875 - dense_2_acc_15: 0.2969 - dense_2_acc_16: 0.3125 - dense_2_acc_17: 0.2031 - dense_2_acc_18: 0.3281 - dense_2_acc_19: 0.1406 - dense_2_acc_20: 0.1719 - dense_2_acc_21: 0.2188 - dense_2_acc_22: 0.1719 - dense_2_acc_23: 0.1406 - dense_2_acc_24: 0.1562 - dense_2_acc_25: 0.1250 - dense_2_acc_26: 0.1094 - dense_2_acc_27: 0.2812 - dense_2_acc_28: 0.3750 - dense_2_acc_29: 0.1562 - dense_2_acc_30: 0.2188 - dense_2_acc_31: 0.1406 - dense_2_acc_32: 0.2031 - dense_2_acc_33: 0.2344 - dense_2_acc_34: 0.2188 - dense_2_acc_35: 0.1875 - dense_2_acc_36: 0

1031/1031 [==============================] - 4s 4ms/step - loss: 415.9778 - dense_2_loss: 3.0556 - dense_2_acc: 0.1920 - dense_2_acc_1: 0.1940 - dense_2_acc_2: 0.1920 - dense_2_acc_3: 0.1843 - dense_2_acc_4: 0.1988 - dense_2_acc_5: 0.2017 - dense_2_acc_6: 0.1920 - dense_2_acc_7: 0.1901 - dense_2_acc_8: 0.2502 - dense_2_acc_9: 0.1882 - dense_2_acc_10: 0.1969 - dense_2_acc_11: 0.2561 - dense_2_acc_12: 0.2241 - dense_2_acc_13: 0.1950 - dense_2_acc_14: 0.1950 - dense_2_acc_15: 0.2328 - dense_2_acc_16: 0.1959 - dense_2_acc_17: 0.2008 - dense_2_acc_18: 0.1872 - dense_2_acc_19: 0.1901 - dense_2_acc_20: 0.1872 - dense_2_acc_21: 0.1969 - dense_2_acc_22: 0.1988 - dense_2_acc_23: 0.1775 - dense_2_acc_24: 0.1930 - dense_2_acc_25: 0.1911 - dense_2_acc_26: 0.1920 - dense_2_acc_27: 0.2541 - dense_2_acc_28: 0.4316 - dense_2_acc_29: 0.1901 - dense_2_acc_30: 0.1882 - dense_2_acc_31: 0.1930 - dense_2_acc_32: 0.1959 - dense_2_acc_33: 0.1950 - dense_2_acc_34: 0.1940 - dense_2_acc_35: 0.1920 - dense_2_acc_3

# Use Trained Weights for Text Generation

In [16]:
def one_hot(softmax_v):
    best_idxs = K.argmax(softmax_v, axis=-1)
    return K.one_hot([best_idxs], num_classes=n_values)

def generate_text(Ty = 140):
    x0 = Input(shape=(1, n_values))
    a0 = Input(shape=(n_a,), name='a0')
    c0 = Input(shape=(n_a,), name='c0')
    a = a0
    c = c0
    x = x0
    outputs = []
    
    for t in range(Ty):
        # Get new activation and cell state
        a, _ , c = LSTM_cell(x, initial_state=[a, c])
        # Use Dense layer to get softmax output
        out = densor(a)
        outputs.append(out)
        
        x = Lambda(one_hot)(out)
    
    return Model([x0, a0, c0], outputs)
    

In [17]:
text_model = generate_text()

In [18]:
# x_initializer = np.zeros((1, 1, n_values))
# a_initializer = np.zeros((1, n_a))
# c_initializer = np.zeros((1, n_a))

x_initializer = np.random.rand(1, 1, n_values)
a_initializer = np.random.rand(1, n_a)
c_initializer = np.random.rand(1, n_a)

In [19]:
def predict_and_sample(x_initializer, a_initializer, c_initializer):
    pred = text_model.predict([x_initializer, a_initializer, c_initializer])
    indices = np.argmax(pred, axis=-1)
    results = to_categorical(indices, num_classes=n_values)
    return results, indices

In [20]:
results, indices = predict_and_sample(x_initializer, a_initializer, c_initializer)

In [21]:
new_chars = [idx_to_char[str(i)] for i in np.ndarray.flatten(indices)]

In [22]:
''.join(new_chars)

'                                                                                                                                            '

In [23]:
indices

array([[0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
    

In [24]:
pred = text_model.predict([x_initializer, a_initializer, c_initializer])

In [25]:
pred[0]

array([[9.3824506e-01, 4.2536175e-09, 1.1826774e-12, 7.8764675e-13,
        1.3358439e-12, 1.5777747e-05, 4.0121037e-08, 2.5463493e-07,
        2.6969171e-10, 1.3511743e-10, 1.7447981e-10, 7.1990523e-14,
        9.3583676e-14, 1.0355450e-13, 2.2164736e-14, 5.6272503e-03,
        1.6863752e-06, 1.2896900e-05, 3.1579455e-04, 3.5612017e-02,
        7.8137846e-06, 1.3964811e-05, 1.9992075e-03, 2.0477262e-03,
        3.1591785e-11, 5.2087347e-07, 2.7033142e-04, 7.9225711e-06,
        1.5104307e-03, 2.0760435e-03, 1.4162144e-06, 1.8143549e-10,
        6.1894354e-04, 8.8788010e-04, 1.0621615e-02, 6.3921085e-05,
        1.2604825e-07, 2.8353019e-05, 5.2916779e-11, 9.2146438e-06,
        2.3830441e-12, 2.7441035e-07, 3.5597786e-06, 1.0364882e-12,
        1.6412555e-12]], dtype=float32)